# Inbound Notebook

This notebook is designed to semi-automate the reporting process for the Inbound team. It will streamline data extraction, transformation, and loading into a pre-formatted Excel file.

## Manual Preparation

The first step involves manually preparing the data in Excel:

1. **Filter the Pivot Table:**
   - Apply filters to the pivot table to extract the following categories:
     - Active
     - Canceled
     - Pending Signature
     - Net

2. **Create Separate Sheets:**
   - For each category (Active, Canceled, Pending Signature, Net), create a separate sheet in the Excel file containing the filtered data.

3. **Save the Excel File:**
   - Save the prepared Excel file with a specific name, ensuring it contains the sheets with the filtered data.

4. **Upload the Excel File:**
   - Upload the prepared Excel file to the designated directory.

## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [7]:
# Import necessary libraries
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [8]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'

# Range to read (change this for each project)
start_row = 8
end_row = 65
usecols = 'A:AF'

print("Variables defined correctly!")

Variables defined correctly!


## Extract Data from "Informe de métricas históricas" CSV file
Extract data from the CSV file and convert it directly to a DataFrame.

In [20]:
df = pd.read_csv(csv_file_path)

pivot_df = df.pivot_table(index=['Agent', 'StartInterval', 'EndInterval'], 
                          values=['Contacts handled incoming', 'Contacts transferred out internal'], 
                          aggfunc='first') 


print(pivot_df)

                                                                                                Contacts handled incoming  \
Agent                              StartInterval                 EndInterval                                                
Irati.izaguirre@originaltelecom.es 2024-08-01T00:00:00.000+02:00 2024-08-02T00:00:00.000+02:00                       19.0   
                                   2024-08-02T00:00:00.000+02:00 2024-08-03T00:00:00.000+02:00                       23.0   
                                   2024-08-05T00:00:00.000+02:00 2024-08-06T00:00:00.000+02:00                       38.0   
                                   2024-08-06T00:00:00.000+02:00 2024-08-07T00:00:00.000+02:00                       43.0   
                                   2024-08-07T00:00:00.000+02:00 2024-08-08T00:00:00.000+02:00                       33.0   
...                                                                                                                   ...   


## Extract Data from 'Active' Sheet

Extract data from the "Active" sheet within the specified range and convert it directly to a DataFrame.

## Verify Columns in 'Active' Sheet

Verify the number of columns in the "Active" sheet to ensure the range is within bounds.

In [10]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

Max column in 'Active' sheet: 10
Adjusted usecols to: A:J


## Extract Data from 'Active' Sheet

Extract data from the "Active" sheet within the specified range and convert it directly to a DataFrame.

In [11]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)

Data from 'ACTIVOS' sheet loaded successfully.


## Extract Data from 'Canceled' Sheet

Extract data from the "Canceled" sheet within the specified range and convert it directly to a DataFrame.

In [12]:
# Extract data from 'Canceled' sheet
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)

Data from 'CANCELADOS' sheet loaded successfully.


## Extract Data from 'Pending Signature' Sheet

Extract data from the "Pending Signature" sheet within the specified range and convert it directly to a DataFrame.

In [13]:
# Extract data from 'Pending Signature' sheet
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)

Data from 'PTE FIRMA' sheet loaded successfully.


## Display DataFrames

Display the first few rows of each DataFrame to verify the data.

In [14]:
# Display the DataFrames
print("Active DataFrame:")
display(active_df.head())

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

Active DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,339.0,296.0,189.0,122.0,405.0,443.0,457.0,351.0,2602
1,david.molero@originaltelecom.es,9.0,5.0,NaN,NaN,18.0,24.0,26.0,27.0,109
2,francisco.perdomo@originaltelecom.es,17.0,13.0,NaN,NaN,18.0,19.0,22.0,18.0,107
3,albertosanchez@originaltelecom.es,15.0,10.0,NaN,NaN,13.0,16.0,35.0,18.0,107
4,miguel.segura@originaltelecom.es,17.0,16.0,NaN,NaN,15.0,17.0,26.0,6.0,97


Canceled DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,132.0,117.0,56.0,47.0,112.0,62.0,42.0,28.0,596
1,antonio.reina@originaltelecom.es,1.0,12.0,NaN,NaN,9.0,10.0,2.0,NaN,34
2,francisco.perdomo@originaltelecom.es,11.0,8.0,NaN,NaN,7.0,2.0,1.0,NaN,29
3,guillermo.hurtado@originaltelecom.es,6.0,10.0,5.0,1.0,NaN,NaN,6.0,NaN,28
4,anasanchez@originaltelecom.es,NaN,NaN,6.0,4.0,12.0,2.0,1.0,NaN,25


Pending Signature DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,20.0,13.0,1.0,1.0,9.0,18.0,26.0,73.0,161
1,david.molero@originaltelecom.es,NaN,2.0,NaN,NaN,NaN,3.0,1.0,8.0,14
2,miriam.rodriguez@originaltelecom.es,NaN,3.0,NaN,NaN,NaN,NaN,NaN,10.0,13
3,miguel.segura@originaltelecom.es,NaN,3.0,NaN,NaN,NaN,5.0,2.0,2.0,12
4,mariaarroyo@originaltelecom.es,2.0,NaN,NaN,NaN,5.0,1.0,1.0,1.0,10


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [15]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

## Apply Transformation

Apply the transformation to replace NaN values with 0 in each DataFrame.

In [16]:
# Apply the transformation
active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)

# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head())

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,339.0,296.0,189.0,122.0,405.0,443.0,457.0,351.0,2602
1,david.molero@originaltelecom.es,9.0,5.0,0.0,0.0,18.0,24.0,26.0,27.0,109
2,francisco.perdomo@originaltelecom.es,17.0,13.0,0.0,0.0,18.0,19.0,22.0,18.0,107
3,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,13.0,16.0,35.0,18.0,107
4,miguel.segura@originaltelecom.es,17.0,16.0,0.0,0.0,15.0,17.0,26.0,6.0,97


Canceled DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,132.0,117.0,56.0,47.0,112.0,62.0,42.0,28.0,596
1,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,2.0,0.0,34
2,francisco.perdomo@originaltelecom.es,11.0,8.0,0.0,0.0,7.0,2.0,1.0,0.0,29
3,guillermo.hurtado@originaltelecom.es,6.0,10.0,5.0,1.0,0.0,0.0,6.0,0.0,28
4,anasanchez@originaltelecom.es,0.0,0.0,6.0,4.0,12.0,2.0,1.0,0.0,25


Pending Signature DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,20.0,13.0,1.0,1.0,9.0,18.0,26.0,73.0,161
1,david.molero@originaltelecom.es,0.0,2.0,0.0,0.0,0.0,3.0,1.0,8.0,14
2,miriam.rodriguez@originaltelecom.es,0.0,3.0,0.0,0.0,0.0,0.0,0.0,10.0,13
3,miguel.segura@originaltelecom.es,0.0,3.0,0.0,0.0,0.0,5.0,2.0,2.0,12
4,mariaarroyo@originaltelecom.es,2.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,10


## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [17]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'carmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'pilar.deval@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mauricio.pozo@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'lailasetati@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'mar.marchena@originaltelecom

## Reorganize and Group Data in Informe.csv

Transform the CSV data into a more structured format by pivoting the DataFrame. This step groups the data by agent and date, allowing for easy access to each agent's daily call statistics. The resulting structure organizes call data by date for each agent, creating a clear and manageable dataset for further processing.


In [19]:
df = pd.read_csv(csv_file_path)

pivot_df = df.pivot_table(index=['Agente', 'Fecha'], 
                          values=['Llamadas Entrantes', 'Llamadas Salientes'], 
                          aggfunc='sum')

print(pivot_df)

KeyError: 'Llamadas Entrantes'

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [18]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)
informe_df = ensure_all_agents(informe_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head())

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head())

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head())

print("Informe DataFrame after ensuring all agents:")
display(informe_df.head(55))

Added 3 missing agents and removed 15 agents not in the list.
Added 5 missing agents and removed 15 agents not in the list.
Added 14 missing agents and removed -3 agents not in the list.
Added 10 missing agents and removed 46 agents not in the list.
Active DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
1,david.molero@originaltelecom.es,9.0,5.0,0.0,0.0,18.0,24.0,26.0,27.0,109
2,francisco.perdomo@originaltelecom.es,17.0,13.0,0.0,0.0,18.0,19.0,22.0,18.0,107
3,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,13.0,16.0,35.0,18.0,107
4,miguel.segura@originaltelecom.es,17.0,16.0,0.0,0.0,15.0,17.0,26.0,6.0,97
5,gonzalofalcon@originaltelecom.es,15.0,13.0,0.0,0.0,12.0,15.0,23.0,19.0,97


Canceled DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
1,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,2.0,0.0,34
2,francisco.perdomo@originaltelecom.es,11.0,8.0,0.0,0.0,7.0,2.0,1.0,0.0,29
3,guillermo.hurtado@originaltelecom.es,6.0,10.0,5.0,1.0,0.0,0.0,6.0,0.0,28
4,anasanchez@originaltelecom.es,0.0,0.0,6.0,4.0,12.0,2.0,1.0,0.0,25
5,manuelvaldes@originaltelecom.es,11.0,3.0,0.0,0.0,6.0,4.0,0.0,0.0,24


Pending Signature DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
1,david.molero@originaltelecom.es,0.0,2.0,0.0,0.0,0.0,3.0,1.0,8.0,14
2,miriam.rodriguez@originaltelecom.es,0.0,3.0,0.0,0.0,0.0,0.0,0.0,10.0,13
3,miguel.segura@originaltelecom.es,0.0,3.0,0.0,0.0,0.0,5.0,2.0,2.0,12
4,mariaarroyo@originaltelecom.es,2.0,0.0,0.0,0.0,5.0,1.0,1.0,1.0,10
5,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,9


Informe DataFrame after ensuring all agents:


,Agent,StartInterval,EndInterval,Contacts handled incoming,Contacts transferred out internal
9,albaaraujo@originaltelecom.es,2024-08-01T00:00:00.000+02:00,2024-08-02T00:00:00.000+02:00,33.0,18.0
10,albaaraujo@originaltelecom.es,2024-08-02T00:00:00.000+02:00,2024-08-03T00:00:00.000+02:00,29.0,8.0
11,albaaraujo@originaltelecom.es,2024-08-05T00:00:00.000+02:00,2024-08-06T00:00:00.000+02:00,63.0,37.0
12,albaaraujo@originaltelecom.es,2024-08-06T00:00:00.000+02:00,2024-08-07T00:00:00.000+02:00,46.0,24.0
13,albaaraujo@originaltelecom.es,2024-08-07T00:00:00.000+02:00,2024-08-08T00:00:00.000+02:00,58.0,34.0
14,albaaraujo@originaltelecom.es,2024-08-08T00:00:00.000+02:00,2024-08-09T00:00:00.000+02:00,59.0,33.0
15,albertocanto@originaltelecom.es,2024-08-01T00:00:00.000+02:00,2024-08-02T00:00:00.000+02:00,36.0,25.0
16,albertocanto@originaltelecom.es,2024-08-02T00:00:00.000+02:00,2024-08-03T00:00:00.000+02:00,16.0,9.0
17,albertocanto@originaltelecom.es,2024-08-05T00:00:00.000+02:00,2024-08-06T00:00:00.000+02:00,64.0,44.0
18,albertocanto@originaltelecom.es,2024-08-06T00:00:00.000+02:00,2024-08-07T00:00:00.000+02:00,42.0,27.0


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [38]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'carmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'dolores.cortes@originaltelecom.es': 22,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'pilar.deval@originaltelecom.es': 39,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mauricio.pozo@originaltelecom.es': 30,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'lailasetati@originaltelecom.es': 20,
    'laura.eguens@originaltelecom.es': 19,
    'leonor.lopez@originaltelecom.es': 21,
    'manuelvaldes@originaltelecom.es': 23,
    'mar.marchena@originaltelecom.es': 33,
    'maria.torres@originaltelecom.es': 28,
    'mariaarroyo@originaltelecom.es': 27,
    'mariangeles.bueso@originaltelecom.es': 26,
    'marta.dorado@originaltelecom.es': 29,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)
informe_df = add_email_values(informe_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head())

print("Canceled DataFrame with email values:")
display(canceled_df.head())

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head())

print("Informe DataFrame with email values:")
display(informe_df.head(65))

Active DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
17,albaaraujo@originaltelecom.es,10.0,9.0,0.0,0.0,9.0,27.0,9.0,6.0,70,1
20,albertocanto@originaltelecom.es,9.0,7.0,0.0,0.0,12.0,9.0,19.0,11.0,67,2
3,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,13.0,16.0,35.0,18.0,107,3
13,anasanchez@originaltelecom.es,0.0,0.0,20.0,15.0,16.0,10.0,19.0,0.0,80,4
25,antonio.reina@originaltelecom.es,10.0,5.0,0.0,0.0,5.0,6.0,12.0,13.0,51,5


Canceled DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
38,albaaraujo@originaltelecom.es,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,5,1
18,albertocanto@originaltelecom.es,0.0,7.0,0.0,0.0,0.0,4.0,0.0,2.0,13,2
22,albertosanchez@originaltelecom.es,4.0,1.0,0.0,0.0,2.0,3.0,3.0,0.0,13,3
4,anasanchez@originaltelecom.es,0.0,0.0,6.0,4.0,12.0,2.0,1.0,0.0,25,4
1,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,2.0,0.0,34,5


Pending Signature DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
37,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1
5,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,9,2
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,9,3
40,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4
8,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,8,5


Informe DataFrame with email values:


,Agent,Contacts handled incoming,Contacts transferred out internal,email_value
2,albaaraujo@originaltelecom.es,59,33,1
3,albertocanto@originaltelecom.es,41,20,2
4,albertosanchez@originaltelecom.es,53,20,3
5,antonio.reina@originaltelecom.es,31,16,5
6,beatriz.gomez@originaltelecom.es,31,18,7
7,carolinafuentes@originaltelecom.es,23,13,9
8,david.molero@originaltelecom.es,59,26,11
9,dolores.cortes@originaltelecom.es,37,22,22
10,elenaborrero@originaltelecom.es,63,30,12
13,francisco.perdomo@originaltelecom.es,46,24,14


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [40]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)
informe_df = sort_by_email_value(informe_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head())  # Displaying first 20 rows for testing

print("Sorted Canceled DataFrame:")
display(canceled_df.head())  # Displaying first 20 rows for testing

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head())  # Displaying first 20 rows for testing

print("Sorted Informe DataFrame:")
display(informe_df.head(65))  # Displaying first 20 rows for testing

Sorted Active DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
17,albaaraujo@originaltelecom.es,10.0,9.0,0.0,0.0,9.0,27.0,9.0,6.0,70,1
20,albertocanto@originaltelecom.es,9.0,7.0,0.0,0.0,12.0,9.0,19.0,11.0,67,2
3,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,13.0,16.0,35.0,18.0,107,3
13,anasanchez@originaltelecom.es,0.0,0.0,20.0,15.0,16.0,10.0,19.0,0.0,80,4
25,antonio.reina@originaltelecom.es,10.0,5.0,0.0,0.0,5.0,6.0,12.0,13.0,51,5


Sorted Canceled DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
38,albaaraujo@originaltelecom.es,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,5,1
18,albertocanto@originaltelecom.es,0.0,7.0,0.0,0.0,0.0,4.0,0.0,2.0,13,2
22,albertosanchez@originaltelecom.es,4.0,1.0,0.0,0.0,2.0,3.0,3.0,0.0,13,3
4,anasanchez@originaltelecom.es,0.0,0.0,6.0,4.0,12.0,2.0,1.0,0.0,25,4
1,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,2.0,0.0,34,5


Sorted Pending Signature DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general,email_value
37,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1
5,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,9,2
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,9,3
40,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4
8,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,8,5


Sorted Informe DataFrame:


,Agent,Contacts handled incoming,Contacts transferred out internal,email_value
2,albaaraujo@originaltelecom.es,59,33,1
3,albertocanto@originaltelecom.es,41,20,2
4,albertosanchez@originaltelecom.es,53,20,3
26,anasanchez@originaltelecom.es,0,0,4
5,antonio.reina@originaltelecom.es,31,16,5
27,azahara.garcia@originaltelecom.es,0,0,6
6,beatriz.gomez@originaltelecom.es,31,18,7
28,carmen.cornejo@originaltelecom.es,0,0,8
7,carolinafuentes@originaltelecom.es,23,13,9
29,cesar.arnaldo@originaltelecom.es,0,0,10


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [41]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)
informe_df = remove_email_value_column(informe_df)

# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head())

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head())

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head())

print("Informe DataFrame after removing 'email_value' column:")
display(informe_df.head(65))

Active DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
17,albaaraujo@originaltelecom.es,10.0,9.0,0.0,0.0,9.0,27.0,9.0,6.0,70
20,albertocanto@originaltelecom.es,9.0,7.0,0.0,0.0,12.0,9.0,19.0,11.0,67
3,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,13.0,16.0,35.0,18.0,107
13,anasanchez@originaltelecom.es,0.0,0.0,20.0,15.0,16.0,10.0,19.0,0.0,80
25,antonio.reina@originaltelecom.es,10.0,5.0,0.0,0.0,5.0,6.0,12.0,13.0,51


Canceled DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
38,albaaraujo@originaltelecom.es,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,5
18,albertocanto@originaltelecom.es,0.0,7.0,0.0,0.0,0.0,4.0,0.0,2.0,13
22,albertosanchez@originaltelecom.es,4.0,1.0,0.0,0.0,2.0,3.0,3.0,0.0,13
4,anasanchez@originaltelecom.es,0.0,0.0,6.0,4.0,12.0,2.0,1.0,0.0,25
1,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,2.0,0.0,34


Pending Signature DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,Total general
37,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
5,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,1.0,0.0,2.0,2.0,9
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,9
40,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,8


Informe DataFrame after removing 'email_value' column:


,Agent,Contacts handled incoming,Contacts transferred out internal
2,albaaraujo@originaltelecom.es,59,33
3,albertocanto@originaltelecom.es,41,20
4,albertosanchez@originaltelecom.es,53,20
26,anasanchez@originaltelecom.es,0,0
5,antonio.reina@originaltelecom.es,31,16
27,azahara.garcia@originaltelecom.es,0,0
6,beatriz.gomez@originaltelecom.es,31,18
28,carmen.cornejo@originaltelecom.es,0,0
7,carolinafuentes@originaltelecom.es,23,13
29,cesar.arnaldo@originaltelecom.es,0,0


## Process Active DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values.

In [16]:
# Calculate sum of numeric values in each row, divided by 2
active_sums = active_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Active' DataFrame:")
print(active_sums.head())

Calculated sums for 'Active' DataFrame:
17     70.0
20     67.0
3     107.0
13     80.0
25     51.0
dtype: float64


## Update finalFile Excel Sheet

Update the "finalFile" Excel sheet with the calculated values from the Active DataFrame.

In [17]:
from openpyxl import load_workbook

# Path to the final file
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'  # Change this to the correct sheet name

def update_final_file(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column F (6th column)
    start_row = 3
    column = 10  # Column J
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}'.")

# Update the final file with the calculated sums
update_final_file(final_file_path, sheet_name, active_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'.


## Process Canceled DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values in column N starting from row 3.

In [18]:
# Calculate sum of numeric values in each row, divided by 2 for 'Canceled' DataFrame
canceled_sums = canceled_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Canceled' DataFrame:")
print(canceled_sums.head())

Calculated sums for 'Canceled' DataFrame:
38     5.0
18    13.0
22    13.0
4     25.0
1     34.0
dtype: float64


## Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column N.

In [19]:
def update_final_file_canceled(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column N (14th column)
    start_row = 3
    column = 11  # Column K
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Canceled data.")

# Update the final file with the calculated sums for Canceled
update_final_file_canceled(final_file_path, sheet_name, canceled_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Canceled data.


## Process Pending Signature DataFrame

Sum all numeric values in each row, divide the result by 2, and store the final values in column Q starting from row 3.

In [20]:
# Calculate sum of numeric values in each row, divided by 2 for 'Pending Signature' DataFrame
pending_signature_sums = pending_signature_df.iloc[:, 1:].sum(axis=1) / 2  # Assuming the first column is not numeric
print("Calculated sums for 'Pending Signature' DataFrame:")
print(pending_signature_sums.head())

Calculated sums for 'Pending Signature' DataFrame:
37    1.0
5     9.0
6     9.0
40    0.0
8     8.0
dtype: float64


## Update finalFile Excel Sheet with Pending Signature Data

Update the "finalFile" Excel sheet with the calculated values from the Pending Signature DataFrame in column Q.

In [21]:
def update_final_file_pending_signature(file_path, sheet_name, values):
    # Load the workbook and select the sheet
    workbook = load_workbook(filename=file_path)
    sheet = workbook[sheet_name]
    
    # Start updating from row 3 in column Q (17th column)
    start_row = 3
    column = 12  # Column L
    
    for i, value in enumerate(values, start=start_row):
        sheet.cell(row=i, column=column, value=value)
    
    # Save the workbook
    workbook.save(file_path)
    print(f"Updated {len(values)} rows in '{sheet_name}' sheet of '{file_path}' with Pending Signature data.")

# Update the final file with the calculated sums for Pending Signature
update_final_file_pending_signature(final_file_path, sheet_name, pending_signature_sums)

Updated 43 rows in 'GLOBAL AGOSTO' sheet of '/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx' with Pending Signature data.


## Update finalFile Excel Sheet with Informe Data

Update the "finalFile" Excel sheet with the raw values from the Informe DataFrame in column Q.

## Define Useful Columns for Active DataFrame

Determine the range of columns that will be used for the Active DataFrame.

In [22]:
# Determine the range of columns for Active DataFrame
active_usecols = active_df.columns[:-1]  # Exclude the last column
print("Active DataFrame Useful Columns:")
print(active_usecols)

Active DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00],
      dtype='object')


## Define Useful Columns for Canceled DataFrame

Determine the range of columns that will be used for the Canceled DataFrame.

In [23]:
# Determine the range of columns for Canceled DataFrame
canceled_usecols = canceled_df.columns[:-1]  # Exclude the last column
print("Canceled DataFrame Useful Columns:")
print(canceled_usecols)

Canceled DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00],
      dtype='object')


## Define Useful Columns for Pending Signature DataFrame

Determine the range of columns that will be used for the Pending Signature DataFrame.

In [24]:
# Determine the range of columns for Pending Signature DataFrame
pending_signature_usecols = pending_signature_df.columns[:-1]  # Exclude the last column
print("Pending Signature DataFrame Useful Columns:")
print(pending_signature_usecols)

Pending Signature DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00],
      dtype='object')


## Update Diario Agosto Sheet with Active DataFrame

For each useful column in the Active DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [25]:
def update_diario_agosto_with_last_column(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Active DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Active DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Active DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'C',   # Assuming 'B' is the first column
        3: 'H',   # Assuming 'C' is the second column
        4: 'M',   # Assuming 'D' is the third column
        5: 'R',   # Assuming 'E' is the fourth column
        6: 'AB',  # Assuming 'F' is the fifth column
        7: 'AG',
        8: 'AL',
        9: 'AQ',
        10: 'AV',
        11: 'BA',
        12: 'BF',
        13: 'BK',
        14: 'BP',
        15: 'BU',
        16: 'BZ',
        17: 'CE',
        18: 'CJ',
        19: 'CO',
        20: 'CT',
        21: 'CY',
        22: 'DD',
        23: 'DI',
        24: 'DN',
        25: 'DS',
        26: 'DX',
        27: 'EC',
        28: 'EH',
        29: 'EM',
        30: 'ER',
        31: 'EW'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Active DataFrame.")

# Update the Diario Agosto sheet with the last column of the Active DataFrame
update_diario_agosto_with_last_column(active_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-08 00:00:00' from Active DataFrame.


## Update Diario Agosto Sheet with Canceled DataFrame

For each useful column in the Canceled DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [26]:
def update_diario_agosto_with_last_column_canceled(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Canceled DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Canceled DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Canceled DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'D',   
        3: 'I',   
        4: 'N',   
        5: 'S',   
        6: 'AC',  
        7: 'AH',
        8: 'AM',
        9: 'AR',
        10: 'AW',
        11: 'BB',
        12: 'BG',
        13: 'BL',
        14: 'BQ',
        15: 'BV',
        16: 'CA',
        17: 'CF',
        18: 'CK',
        19: 'CP',
        20: 'CU',
        21: 'CZ',
        22: 'DE',
        23: 'DJ',
        24: 'DO',
        25: 'DT',
        26: 'DY',
        27: 'ED',
        28: 'EI',
        29: 'EN',
        30: 'ES',
        31: 'EX'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Canceled DataFrame.")

# Update the Diario Agosto sheet with the last column of the Canceled DataFrame
update_diario_agosto_with_last_column_canceled(canceled_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-08 00:00:00' from Canceled DataFrame.


## Update Diario Agosto Sheet with Pending Signature DataFrame

For each useful column in the Pending Signature DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [27]:
def update_diario_agosto_with_last_column_pending_signature(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Pending Signature DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Pending Signature DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Pending Signature DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'E',   # Adjusted to the next column
        3: 'J',   
        4: 'O',   
        5: 'T',   
        6: 'AD',  
        7: 'AI',
        8: 'AN',
        9: 'AS',
        10: 'AX',
        11: 'BC',
        12: 'BH',
        13: 'BM',
        14: 'BR',
        15: 'BW',
        16: 'CB',
        17: 'CG',
        18: 'CL',
        19: 'CQ',
        20: 'CV',
        21: 'DA',
        22: 'DF',
        23: 'DK',
        24: 'DP',
        25: 'DU',
        26: 'DZ',
        27: 'EE',
        28: 'EJ',
        29: 'EO',
        30: 'ET',
        31: 'EY'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EY')  # Default to 'EY' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Pending Signature DataFrame.")

# Update the Diario Agosto sheet with the last column of the Pending Signature DataFrame
update_diario_agosto_with_last_column_pending_signature(pending_signature_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-08 00:00:00' from Pending Signature DataFrame.


## Update Diario Agosto Sheet with Informe DataFrame

For each useful column in the Informe DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.